In [ ]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [12]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.io import read_image
import timm   

from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

In [13]:
DATA_DIR = "/kaggle/input/raf-db-dataset/DATASET"
TRAIN_CSV = "/kaggle/input/raf-db-dataset/train_labels.csv"
VAL_CSV   = "/kaggle/input/raf-db-dataset/test_labels.csv"


In [ ]:
DATA_DIR = "/kaggle/input/raf-db-dataset/DATASET"
TRAIN_CSV = "/kaggle/input/raf-db-dataset/train_labels.csv"
VAL_CSV   = "/kaggle/input/raf-db-dataset/test_labels.csv"

OUTPUT_DIR = "./outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

BATCH_SIZE = 32
IMAGE_SIZE = 224
NUM_CLASSES = 7  
LR = 3e-4
EPOCHS = 20
PATIENCE = 3
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [28]:
class RAFDBDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data.iloc[idx, 0]      
        label = int(self.data.iloc[idx, 1])    

        
        img_path = os.path.join(self.root_dir, str(label), img_name)

        image = read_image(img_path).float() / 255.0

        if self.transform:
            image = self.transform(image)

        return image, label-1  

In [16]:
train_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

val_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.Normalize(mean=[0.5], std=[0.5])
])



In [ ]:
train_ds = RAFDBDataset(
    TRAIN_CSV, 
    os.path.join(DATA_DIR, "train"), 
    transform=train_transform
)

val_ds = RAFDBDataset(
    VAL_CSV, 
    os.path.join(DATA_DIR, "test"), 
    transform=val_transform
)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)


In [ ]:
train_labels = pd.read_csv(TRAIN_CSV)['label'].astype(int).values
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(train_labels),
                                     y=train_labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(DEVICE)
print("Class Weights:", class_weights)

In [ ]:
model = timm.create_model("convnext_tiny", pretrained=True, num_classes=NUM_CLASSES)
model = model.to(DEVICE)

In [20]:
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [ ]:
best_val_f1 = 0.0
patience_counter = 0

for epoch in range(1, EPOCHS+1):
   
    model.train()
    train_loss, correct, total = 0.0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    train_loss /= total

    
    model.eval()
    val_loss, correct, total = 0.0, 0, 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_acc = correct / total
    val_loss /= total

    report = classification_report(all_labels, all_preds, digits=4, zero_division=0, output_dict=True)
    val_f1 = report['weighted avg']['f1-score']

   
    print(f"Epoch {epoch}/{EPOCHS} | "
          f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f} F1: {val_f1:.4f}")

    scheduler.step()

   
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model.state_dict(), os.path.join(OUTPUT_DIR, "best_model.pth"))
        print(" Saved Best Model")
        patience_counter = 0
    else:
        patience_counter += 1

   
    if patience_counter >= PATIENCE:
        print(" Early Stopping triggered")
        break

print("Training finished. Best F1:", best_val_f1)

In [33]:
torch.save(model.state_dict(), os.path.join(OUTPUT_DIR, "best_model.pth"))


In [ ]:
model.load_state_dict(torch.load(os.path.join(OUTPUT_DIR, "best_model.pth")))
model.eval()


In [ ]:
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        preds = outputs.argmax(dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

from sklearn.metrics import classification_report
print(classification_report(all_labels, all_preds, digits=4))
